In [1]:
import os
import json

coco_train_dir = "train/"
yolo_output_dir = "self_driving_yolo"

In [2]:
coco_labels_json = os.path.join(coco_train_dir, "labels.json")

In [3]:
assert os.path.exists(coco_labels_json)

In [4]:
with open(coco_labels_json) as f:
    coco_labels = json.load(f)

In [5]:
coco_labels.keys()

dict_keys(['info', 'licenses', 'categories', 'images', 'annotations'])

In [6]:
yolo_config = {
    # dataset root dir (absolute or relative; if relative, it's relative to default datasets_dir)
    "path": "self_driving_yolo", 
    "train": "images/train", # train images (relative to 'path') 4 images
    "val": "images/val", # val images (relative to 'path') 4 images
    "names": []
}

In [7]:
categories = {
    i: category['name']
    for i, category in enumerate(coco_labels['categories'][1: ])
}

In [8]:
categories

{0: 'bicycle',
 1: 'bus',
 2: 'car',
 3: 'human',
 4: 'motorcycle',
 5: 'trafficcone',
 6: 'truck'}

In [9]:
categories

{0: 'bicycle',
 1: 'bus',
 2: 'car',
 3: 'human',
 4: 'motorcycle',
 5: 'trafficcone',
 6: 'truck'}

In [10]:
yolo_config['names'] = categories

In [11]:
yolo_config

{'path': 'self_driving_yolo',
 'train': 'images/train',
 'val': 'images/val',
 'names': {0: 'bicycle',
  1: 'bus',
  2: 'car',
  3: 'human',
  4: 'motorcycle',
  5: 'trafficcone',
  6: 'truck'}}

In [12]:
os.makedirs(yolo_output_dir, exist_ok=True)

In [13]:
import yaml
import shutil

In [14]:
with open(os.path.join(yolo_output_dir, "dataset.yaml"), 'w') as f:
    yaml.dump(yolo_config, f)

In [15]:
from collections import defaultdict
id_to_annotations = defaultdict(list)
for ann in coco_labels['annotations']:
    id_to_annotations[ann['image_id']].append(ann)

In [16]:
from util import process_annotation
set_type = 'train'

In [17]:
origin_images_dir = os.path.join(coco_train_dir, "data")

In [18]:
images_dir = os.path.join(yolo_config['path'], "images", set_type)
labels_dir = os.path.join(yolo_config['path'], "labels", set_type)

In [19]:
images_dir

'self_driving_yolo/images/train'

In [20]:
os.makedirs(images_dir, exist_ok=True)
os.makedirs(labels_dir, exist_ok=True)

In [21]:
images = coco_labels['images']

In [22]:
for image in images[:100]:
    annotations = id_to_annotations[image['id']]
    yolo_annotations = [process_annotation(ann, image) for ann in annotations]
    yolo_annotations_str = "\n".join(yolo_annotations)

    output_image_path = os.path.join(images_dir, image['file_name'])
    output_labels_path = os.path.join(labels_dir, image['file_name'].replace(".jpg", ".txt"))
    
    with open(output_labels_path, 'w') as f:
        f.write(yolo_annotations_str)
        
    input_image_path = os.path.join(origin_images_dir, image['file_name'])
    shutil.copy(input_image_path, output_image_path)

Real time object detection 

In [34]:
import numpy as np

In [35]:
n_classes = len(yolo_config['names'])

In [41]:
labels = np.zeros((len(images), n_classes), dtype=np.int32)

In [42]:
for i, image in enumerate(images):
    anns = id_to_annotations[image['id']]
    for ann in anns:
        labels[i, ann['category_id'] - 1] = 1
        m = min(m, ann['category_id'])

In [43]:
from skmultilearn.model_selection import iterative_train_test_split

In [49]:
indices = np.arange(len(images))[:, np.newaxis]

In [51]:
train_indices, train_labels, valid_indices, valid_labels = iterative_train_test_split(
    indices,
    labels,
    test_size=0.2)


In [58]:
train_labels.mean(axis=0)

array([0.14444182, 0.0730369 , 0.63859981, 0.50832545, 0.14411069,
       0.25681173, 0.2705298 ])

In [57]:
valid_labels.mean(axis=0)

array([0.1444518 , 0.07302999, 0.6385394 , 0.50827736, 0.14407341,
       0.25683474, 0.27055151])

In [59]:
train_images = [images[i] for i in train_indices.ravel()]

In [60]:
valid_images = [images[i] for i in valid_indices.ravel()]